In [114]:
import numpy as np

In [115]:
def GMRes_matrix_free(afun, b, x0, threshold):
    xt = np.zeros(len(x0))
    r0 = b - afun(x0)
    nr0 = np.linalg.norm(r0)
    Q = np.zeros((len(x0), len(x0)))
    H = np.zeros((len(x0), len(x0)))
    Q[:, 0] = r0 / nr0
    for k in range(len(x0)):
        y = afun(Q[:, k])
        for j in range(k + 1):
            H[j][k] = np.dot(Q[:, j], y)
            y = y - np.dot(H[j][k], Q[:, j])
        if k + 1 < len(x0):
            H[k + 1][k] = np.linalg.norm(y)
            if np.abs(H[k + 1][k]) > 1e-16:
                Q[:, k + 1] = y / H[k + 1][k]
            else:
                break
            e1 = np.zeros((k + 1) + 1)
            e1[0] = 1
            H_tilde = H[0:(k + 1) + 1, 0:k + 1]
        else:
            H_tilde = H[0:k + 1, 0:k + 1]
        ck = np.linalg.lstsq(H_tilde, nr0 * e1)[0]
        if k + 1 < len(x0):
            x = x0 + np.dot(Q[:, 0:(k + 1)], ck)
        else:
            x = x0 + np.dot(Q, ck)
        norm_small = np.linalg.norm(np.dot(H_tilde, ck) - nr0 * e1)
        if norm_small < threshold:
            break
    return x 

In [124]:
def find_x(n,l,B_1,SFP,b,delta):
    x0 = np.zeros(n)
    #afun return vector w that corresponds to the product between the matrix A and the vector v
    def afun(v):
        b1 = v
        for i in range(l):
            b1 = np.dot(B_1, b1) + SFP(b1)
        return b1
    threshold = delta 
    x_approx = GMRes_matrix_free(afun, b, x0, threshold)
    return x_approx

Este código simula la solución al siguiente problema
$$ (A)^{l}x = b $$

Donde $A$ es una matriz de $n \times n$ y $b$ es un vector de $n \times 1$.

Ademas, $A$ se construye de la siguiente manera: 
$$ B_{1} + B_{2} $$

Siendo $B_{1}$  una matriz sparse y $B_{2}$ una matriz densa de $n \times n$.

El objetivo es resolver el problema solo tienendo acceso a la matriz $B_{1}$, el vector $b$, el exponente $l$ y $SFP(v)$ que es una función que simula el producto matriz-vector de $B_{2}$

In [139]:
# Definir dimensiones
n = 50  # Dimensión del problema
k = 10   # Número máximo de elementos distintos de cero por fila en B1
l = 5    # Exponente de la matriz A

# Generar matriz aleatoria B1
B1 = np.zeros((n, n))
for i in range(n):
    # Generar vector aleatorio de índices
    idx = np.random.choice(n, k, replace=False)
    # Generar vector aleatorio de valores
    vals = np.random.randn(k)
    # Asignar valores a B1
    B1[i, idx] = vals


# Generar matriz B2 (simulación)
B2 = np.random.randn(n, n)

# Generar vector b aleatorio
b = np.random.randn(n)

# Definir delta
delta = 1e-16


def SFP(v):
    return np.dot(B2, v)


# Generar matriz B2 (simulación)
B2 = np.random.randn(n, n)

# Generar vector b aleatorio
b = np.random.randn(n)

# Definir delta
delta = 1e-16

def SFP(v):
    return np.dot(B2, v)


# Encontrar solución
x = find_x(n,l,B1,SFP,b,delta)

# Imprimir solución
print("Solución encontrada:")
print(x)

#armar matriz A
A = B1 + B2
A = np.linalg.matrix_power(A, l)
print("Solución real:")
print(np.linalg.solve(A, b))

#Calcular error
error = np.linalg.norm(x - np.linalg.solve(A, b))
    
print("Error:")
print(error)

Solución encontrada:
[ 0.21372465  0.78860845 -0.55141346 -0.75039288 -0.03524315 -0.0983916
  0.35994629 -0.39114274  0.25396645 -0.02578609 -0.663079   -0.66134821
 -0.57672823 -0.46591763  0.55749026 -0.15124978 -0.02643981  0.69383211
 -0.71997953 -0.54661962 -0.32273848 -0.3264034   0.26089507  1.21902421
  0.37489504 -1.49989592 -1.99686771  1.24496069  0.11049341 -0.45201151
  1.35608079  0.82826845 -0.35797778 -1.19739614  1.08842193 -0.4205288
 -0.49025229  0.47786459 -0.57255784 -0.35951668 -0.30280264  0.23893515
 -0.81758378 -0.14423217 -1.24671749 -0.27783712 -0.06660569 -0.62155251
 -0.02247946  0.57268424]
Solución real:
[ 0.21372465  0.78860845 -0.55141346 -0.75039288 -0.03524315 -0.0983916
  0.35994629 -0.39114274  0.25396645 -0.02578609 -0.663079   -0.66134821
 -0.57672823 -0.46591763  0.55749026 -0.15124978 -0.02643981  0.69383211
 -0.71997953 -0.54661962 -0.32273848 -0.3264034   0.26089507  1.21902421
  0.37489504 -1.49989592 -1.99686771  1.24496069  0.11049341 -0.4

C:\Users\umbin\AppData\Local\Temp\ipykernel_17284\1081799002.py:24: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  ck = np.linalg.lstsq(H_tilde, nr0 * e1)[0]
